In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [5]:
import pyodbc

try:
    conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};"
                          "SERVER=LAPTOP-RES6G37K;"
                          "DATABASE=adventureworks2019;"
                          "Trusted_connection=Yes;")
    print("✅ SQL Server Login Successful!")
    conn.close()
except Exception as e:
    print("❌ SQL Server Login Failed:", e)

✅ SQL Server Login Successful!


In [6]:
from sqlalchemy import create_engine

In [7]:
from sqlalchemy.engine import URL
server = "LAPTOP-RES6G37K"
database = "adventureworks2019"


connection_url = URL.create(
    "mssql+pyodbc",
    host="tcp:" + server,
    port=1433,
    database=database,
    
    # driver is typically required in the query
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "Trusted_Connection": "yes"
    },
)

engine = create_engine(connection_url)


In [8]:
try:
    with engine.connect() as conn:
        print("✅ SQLAlchemy connected successfully!")
except Exception as e:
    print("❌ SQLAlchemy connection failed:", e)

✅ SQLAlchemy connected successfully!


In [9]:
from langchain.utilities import SQLDatabase

In [10]:
from langchain_community.utilities.sql_database import SQLDatabase

In [62]:
from sqlalchemy import create_engine, MetaData

meta = MetaData()
# Reflect *all* schemas and tables:
for schema_name in ["Person","dbo","Sales","HumanResources","Production","Purchasing"]:
    meta.reflect(bind=engine, schema=schema_name)



C:\Users\Sreehari\AppData\Local\Temp\ipykernel_15128\3025684154.py:6: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  meta.reflect(bind=engine, schema=schema_name)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_15128\3025684154.py:6: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  meta.reflect(bind=engine, schema=schema_name)
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_15128\3025684154.py:6: SAWarning: Did not recognize type 'hierarchyid' of column 'DocumentNode'
  meta.reflect(bind=engine, schema=schema_name)


In [29]:
dbase = SQLDatabase(engine=engine, schema="Sales", metadata=meta)

In [30]:
dbase.get_table_names()

['CountryRegionCurrency',
 'CreditCard',
 'Currency',
 'CurrencyRate',
 'Customer',
 'PersonCreditCard',
 'SalesOrderDetail',
 'SalesOrderHeader',
 'SalesOrderHeaderSalesReason',
 'SalesPerson',
 'SalesPersonQuotaHistory',
 'SalesReason',
 'SalesTaxRate',
 'SalesTerritory',
 'SalesTerritoryHistory',
 'ShoppingCartItem',
 'SpecialOffer',
 'SpecialOfferProduct',
 'Store']

In [31]:
len(dbase.get_table_info())

21970

In [32]:
dbase.get_table_info()

'\nCREATE TABLE [Sales].[CountryRegionCurrency] (\n\t[CountryRegionCode] NVARCHAR(3) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\t[CurrencyCode] NCHAR(3) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\t[ModifiedDate] DATETIME NOT NULL DEFAULT (getdate()), \n\tCONSTRAINT [PK_CountryRegionCurrency_CountryRegionCode_CurrencyCode] PRIMARY KEY ([CountryRegionCode], [CurrencyCode]), \n\tCONSTRAINT [FK_CountryRegionCurrency_CountryRegion_CountryRegionCode] FOREIGN KEY([CountryRegionCode]) REFERENCES [Person].[CountryRegion] ([CountryRegionCode]), \n\tCONSTRAINT [FK_CountryRegionCurrency_Currency_CurrencyCode] FOREIGN KEY([CurrencyCode]) REFERENCES [Sales].[Currency] ([CurrencyCode])\n)\n\n/*\n3 rows from CountryRegionCurrency table:\nCountryRegionCode\tCurrencyCode\tModifiedDate\nAE\tAED\t2014-02-08 10:17:21.510000\nAR\tARS\t2014-02-08 10:17:21.510000\nAT\tATS\t2014-02-08 10:17:21.510000\n*/\n\n\nCREATE TABLE [Sales].[CreditCard] (\n\t[CreditCardID] INTEGER NOT NULL IDENTITY(1,1), \n

In [60]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql.base import SQLDatabaseChain

# Your SQL Server connection URI
connection_uri = (
    "mssql+pyodbc://@LAPTOP-RES6G37K/adventureworks2019"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

# Pass multiple schemas to include_schemas
db = SQLDatabase.from_uri(
    connection_uri,
    schema="Person"  # Example multiple schemas
)

# Set up LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# Create SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True,return_intermediate_steps=True,)

# Run a natural language question
response = db_chain.invoke("How many persons whose home address is in country France?")



C:\Users\Sreehari\AppData\Roaming\Python\Python311\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  self._metadata.reflect(




> Entering new SQLDatabaseChain chain...
How many persons whose home address is in country France?
SQLQuery:SELECT COUNT(DISTINCT p.[BusinessEntityID])
FROM [Person].[Person] p
JOIN [Person].[BusinessEntityAddress] bea ON p.[BusinessEntityID] = bea.[BusinessEntityID]
JOIN [Person].[Address] a ON bea.[AddressID] = a.[AddressID]
JOIN [Person].[StateProvince] sp ON a.[StateProvinceID] = sp.[StateProvinceID]
JOIN [Person].[CountryRegion] cr ON sp.[CountryRegionCode] = cr.[CountryRegionCode]
WHERE cr.[Name] = 'France'
SQLResult: [(1811,)]
Answer:There are 1811 persons whose home address is in France.
> Finished chain.


In [58]:
len(response['intermediate_steps'][1])

58

In [35]:
# Your SQL Server connection URI
connection_uri = (
    "mssql+pyodbc://@LAPTOP-RES6G37K/adventureworks2019"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

In [36]:
from sqlalchemy import create_engine, inspect

engine = create_engine(connection_uri)
inspector = inspect(engine)

for schema in inspector.get_schema_names():
    print(f"\nSchema: {schema}")
    for table in inspector.get_table_names(schema=schema):
        print(f"  {schema}.{table}")


Schema: db_accessadmin

Schema: db_backupoperator

Schema: db_datareader

Schema: db_datawriter

Schema: db_ddladmin

Schema: db_denydatareader

Schema: db_denydatawriter

Schema: db_owner

Schema: db_securityadmin

Schema: dbo
  dbo.AWBuildVersion
  dbo.DatabaseLog
  dbo.ErrorLog

Schema: guest

Schema: HumanResources
  HumanResources.Department
  HumanResources.Employee
  HumanResources.EmployeeDepartmentHistory
  HumanResources.EmployeePayHistory
  HumanResources.JobCandidate
  HumanResources.Shift

Schema: INFORMATION_SCHEMA

Schema: LAPTOP-RES6G37K\Sreehari

Schema: Person
  Person.Address
  Person.AddressType
  Person.BusinessEntity
  Person.BusinessEntityAddress
  Person.BusinessEntityContact
  Person.ContactType
  Person.CountryRegion
  Person.EmailAddress
  Person.Password
  Person.Person
  Person.PersonPhone
  Person.PhoneNumberType
  Person.StateProvince

Schema: Production
  Production.BillOfMaterials
  Production.Culture
  Production.Document
  Production.Illustration
  P

In [61]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql.base import SQLDatabaseChain

# Your SQL Server connection URI
connection_uri = (
    "mssql+pyodbc://@LAPTOP-RES6G37K/adventureworks2019"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

# Pass multiple schemas to include_schemas
db = SQLDatabase.from_uri(
    connection_uri,
    schema="Person"  # Example multiple schemas
)

# Set up LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# Create SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True,return_intermediate_steps=True,)

# Run a natural language question
response = db_chain.invoke("How many persons whose sales territory group is belongs to Europe?")



C:\Users\Sreehari\AppData\Roaming\Python\Python311\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  self._metadata.reflect(




> Entering new SQLDatabaseChain chain...
How many persons whose sales territory group is belongs to Europe?
SQLQuery:SELECT COUNT(DISTINCT [Person].[BusinessEntityID]) 
FROM [Person].[Person]
JOIN [Person].[BusinessEntityAddress] ON [Person].[BusinessEntityID] = [BusinessEntityAddress].[BusinessEntityID]
JOIN [Person].[Address] ON [BusinessEntityAddress].[AddressID] = [Address].[AddressID]
JOIN [Person].[StateProvince] ON [Address].[StateProvinceID] = [StateProvince].[StateProvinceID]
JOIN [Sales].[SalesTerritory] ON [StateProvince].[TerritoryID] = [SalesTerritory].[TerritoryID]
WHERE [SalesTerritory].[Group] = 'Europe'
SQLResult: [(5506,)]
Answer:There are 5506 persons whose sales territory group belongs to Europe.
> Finished chain.


In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql.base import SQLDatabaseChain

# Your SQL Server connection URI
connection_uri = (
    "mssql+pyodbc://@LAPTOP-RES6G37K/adventureworks2019"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

# Pass multiple schemas to include_schemas
db = SQLDatabase.from_uri(
    connection_uri,
    schema="Person"  # Example multiple schemas
)

# Set up LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# Create SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True,return_intermediate_steps=True,)

# Run a natural language question
response = db_chain.invoke("How many persons whose sales territory group is belongs to Europe?")



In [63]:
all_tables = list(meta.tables.keys())

In [66]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql.base import SQLDatabaseChain

# Your SQL Server connection URI
connection_uri = (
    "mssql+pyodbc://@LAPTOP-RES6G37K/adventureworks2019"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

# Pass multiple schemas to include_schemas
db = SQLDatabase.from_uri(
    connection_uri,
    schema="Person"  # Example multiple schemas
)

# Set up LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# Create SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True,return_intermediate_steps=True,)


C:\Users\Sreehari\AppData\Roaming\Python\Python311\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  self._metadata.reflect(


In [67]:
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine, inspect
from typing import List, Optional, Dict

class SQLDatabaseWithSchemas(SQLDatabase):

    @classmethod
    def from_uri_with_schemas(
        cls,
        database_uri: str,
        include_schemas: Optional[List[str]] = None,
        sample_rows_in_table_info: int = 3,
        custom_table_info: Optional[Dict[str, str]] = None,
    ) -> "SQLDatabaseWithSchemas":
        engine = create_engine(database_uri)
        inspector = inspect(engine)

        if include_schemas is not None:
            all_tables = []
            for schema in include_schemas:
                tables = inspector.get_table_names(schema=schema)
                all_tables.extend([f"{schema}.{table}" for table in tables])
        else:
            # fallback to default behavior (just pull everything)
            all_tables = inspector.get_table_names()

        return cls(
            engine=engine,
            include_tables=all_tables,
            sample_rows_in_table_info=sample_rows_in_table_info,
            custom_table_info=custom_table_info,
        )


In [69]:
db = SQLDatabaseWithSchemas.from_uri_with_schemas(
    database_uri="mssql+pyodbc://@LAPTOP-RES6G37K/Adventureworks2019?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes",
    include_schemas=["dbo", "Sales", "Person"]
)


ValueError: include_tables {'Sales.SalesPerson', 'Sales.CreditCard', 'Person.PhoneNumberType', 'Sales.Customer', 'Person.Address', 'Sales.SpecialOffer', 'Person.EmailAddress', 'Person.AddressType', 'Sales.Store', 'Sales.SalesOrderDetail', 'Sales.ShoppingCartItem', 'Sales.SalesOrderHeaderSalesReason', 'Person.BusinessEntityContact', 'dbo.AWBuildVersion', 'Sales.SalesTerritoryHistory', 'Person.BusinessEntityAddress', 'Person.CountryRegion', 'Sales.SalesTerritory', 'Person.Person', 'Sales.SalesTaxRate', 'Sales.SpecialOfferProduct', 'Person.Password', 'Person.PersonPhone', 'Sales.CurrencyRate', 'Person.StateProvince', 'Sales.SalesReason', 'Sales.CountryRegionCurrency', 'dbo.DatabaseLog', 'Person.ContactType', 'Sales.Currency', 'Sales.PersonCreditCard', 'dbo.ErrorLog', 'Sales.SalesPersonQuotaHistory', 'Sales.SalesOrderHeader', 'Person.BusinessEntity'} not found in database

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import initialize_agent, AgentType

# 1. Connect to SQL Server (example using Windows Auth)
db_uri = (
    "mssql+pyodbc://@LAPTOP-RES6G37K/Adventureworks2019?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)
db = SQLDatabase.from_uri(
    db_uri,
    schema="Person"
)

# 2. Create the LLM and toolkit
llm = ChatOpenAI(temperature=0, model="gpt-4")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# 3. Initialize agent with SQL tools
agent_executor = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # or AgentType.ZERO_SHOT_REACT_DESCRIPTION
    verbose=True
)


C:\Users\Sreehari\AppData\Roaming\Python\Python311\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  self._metadata.reflect(
C:\Users\Sreehari\AppData\Local\Temp\ipykernel_15128\132290372.py:20: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [74]:
question = "How many persons whose home address is in country France?"
response = agent_executor.run(question)

print("\n✅ Final Answer:")
print(response)




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: To answer your question, I need to know the structure of the database. Let's first check the list of tables in the database.

Address, AddressType, BusinessEntity, BusinessEntityAddress, BusinessEntityContact, ContactType, CountryRegion, EmailAddress, Password, Person, PersonPhone, PhoneNumberType, StateProvince
Invoking: `sql_db_schema` with `{'table_names': 'Person, Address, CountryRegion'}`
responded: The database contains several tables. To find the number of persons whose home address is in France, we need to look at the 'Person', 'Address', and 'CountryRegion' tables. Let's check the schema of these tables to understand their structure.


CREATE TABLE [Person].[Address] (
	[AddressID] INTEGER NOT NULL IDENTITY(1,1), 
	[AddressLine1] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[AddressLine2] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[City] NVARCHAR(30) COL